### Imports

In [21]:
import requests, json, time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os, sys

In [34]:
base_url = 'https://www.goodreads.com/book'
url = 'https://www.goodreads.com/book/show/1'

#### Setting up the driver

In [35]:
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver") 
driver.get(url)

In [33]:
soup = BeautifulSoup(driver.page_source, 'lxml')

#### Scrapper for rating

In [32]:
first_url = 8000    # Book number - start point 
last_url = 10000       # Book number - end point (Last book is 8,630,000)
ratings = []          # Empty container to store new columns 
for book_reference_number in range(first_url, last_url):
    print(book_reference_number)  # This prints which book the scrapper is currently scrapping 
    driver.get("https://www.goodreads.com/book/show/"+str(book_reference_number)) # This gets the url for the book 
    time.sleep(2) 
    soup = BeautifulSoup(driver.page_source, 'lxml') #create a soup object
    
    
    no_title = soup.find('title').text
    if no_title == 'Page not found':    
        continue
    

    last_page_source = ''

    while True:
        page_changed = False # It's useful to declare whether the page has changed or not
        attempts = 0
        while(not page_changed):
            if last_page_source != driver.page_source:
                page_changed = True
            else:
                if attempts > 5: # Decide on some point when you want to give up.
                    break;
                else:
                    time.sleep(3) # Give time to load new page. Interval could be shorter.
                    attempts += 1
        if page_changed:
            soup_1 = BeautifulSoup(driver.page_source, 'lxml')
            user = soup_1.find('div', {'id': 'bookReviews'})
            user = user.find_all('div',{'class':'friendReviews elementListBrown'})
            review_list = soup_1.find_all('div', class_ = 'reviewHeader uitext stacked')
            for row in review_list: # finding for each separate rating

                rating = {}
                
                try:
                    book_title = soup.select('.gr-h1.gr-h1--serif')[0].text.strip()
                except:
                    book_title = ''
                    
                #ratings.append({'book_name': book_title, 'book_id': book_reference_number})
                
    
                try:
                    
                    # try and except is needed because not all the users have a rating
                    rating['user_id'] = row.find('a', class_ = 'user', href = True)['href'].split('/')[3].split('-')[0] # grabbing the username
                    rating['rating'] = row.find('span',{'class':'staticStars'})['title'] # grabbing user rating out of 5
                    rating['book_name']= book_title
                    rating['book_id'] = book_reference_number
                    ratings.append(rating)
                    

                except:
                    pass
            last_page_source = driver.page_source
            try:
                next_page_element = driver.find_element_by_class_name('next_page')
                driver.execute_script("arguments[0].click();", next_page_element) # clicking on the next button to scrape the other page
                time.sleep(2)
            except:
                break
        else:
            
            df_rev = pd.DataFrame(ratings) # merging all the results to build a data frame
            #print(df_rev.drop_duplicates())
            break;
            
df_rev.to_csv('./datasets/8000_10000_books.csv')

6500
6501
6502
6503
6504
6505
6506
6507
6508
6509
6510
6511
6512
6513
6514
6515
6516
6517
6518
6519
6520
6521
6522
6523
6524
6525
6526
6527
6528
6529
6530
6531
6532
6533
6534
6535
6536
6537
6538
6539
6540
6541
6542
6543
6544
6545
6546
6547
6548
6549
6550
6551
6552
6553
6554
6555
6556
6557
6558
6559
6560
6561
6562
6563
6564
6565
6566
6567
6568
6569
6570
6571
6572
6573
6574
6575
6576
6577
6578
6579
6580
6581
6582
6583
6584
6585
6586
6587
6588
6589
6590
6591
6592
6593
6594
6595
6596
6597
6598
6599
6600
6601
6602
6603
6604
6605
6606
6607
6608
6609
6610
6611
6612
6613
6614
6615
6616
6617
6618
6619
6620
6621
6622
6623
6624
6625
6626
6627
6628
6629
6630
6631
6632
6633
6634
6635
6636
6637
6638
6639
6640
6641
6642
6643
6644
6645
6646
6647
6648
6649
6650
6651
6652
6653
6654
6655
6656
6657
6658
6659
6660
6661
6662
6663
6664
6665
6666
6667
6668
6669
6670
6671
6672
6673
6674
6675
6676
6677
6678
6679
6680
6681
6682
6683
6684
6685
6686
6687
6688
6689
6690
6691
6692
6693
6694
6695
6696
6697
6698
6699


AttributeError: 'NoneType' object has no attribute 'find_all'

#### Scrapper for book description

In [38]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver

first_url = 1000
last_url = 3000    
container = []
for book_reference_number in range(first_url, last_url):
    print(book_reference_number)
    time.sleep(5)
    driver.get("https://www.goodreads.com/book/show/"+str(book_reference_number))
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    desc_dict = {}
    
    try:
        book_title = soup.select('.gr-h1.gr-h1--serif')[0].text.strip()
    except:
        book_title = ''
#     try:
#         author_name = soup.select('.authorName')[0].text.strip()
#     except:
#         author_name = ''
        time.sleep(1)
    try:
        description_list = soup.find( 'div', class_= 'readable stacked').text
        time.sleep(3)
    except:
        continue
       
        
    desc_dict['description'] = description_list
#     desc_dict['book_name']= book_title
#     desc_dict['author_name'] = author_name
    desc_dict['book_id'] = book_reference_number
    container.append(desc_dict)

    df_desc = pd.DataFrame(container)

df_desc.to_csv('./description/1000_3000_books.csv')    

1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199


2639
2640
2641
2642
2643
2644
2645
2646
2647
2648
2649
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2663
2664
2665
2666
2667
2668
2669
2670
2671
2672
2673
2674
2675
2676
2677
2678
2679
2680
2681
2682
2683
2684
2685
2686
2687
2688
2689
2690
2691
2692
2693
2694
2695
2696
2697
2698
2699
2700
2701
2702
2703
2704
2705
2706
2707
2708
2709
2710
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838


WORKNOTES TO BUILD THE SCRAPPER ABOVE

In [27]:
#find the title of the book along with the author
title = soup.find('title').text
title

'Harry Potter and the Half-Blood Prince by J.K. Rowling'

In [28]:
#find all the ratings of the book
description_list = soup.find( 'div', class_= 'readable stacked')
description_list.text

'\nWhen Harry Potter and the Half-Blood Prince opens, the war against Voldemort has begun. The Wizarding world has split down the middle, and as the casualties mount, the effects even spill over onto the Muggles. Dumbledore is away from Hogwarts for long periods, and the Order of the Phoenix has suffered grievous losses. And yet, as in all wars, life goes on.Harry, Ron, and\nWhen Harry Potter and the Half-Blood Prince opens, the war against Voldemort has begun. The Wizarding world has split down the middle, and as the casualties mount, the effects even spill over onto the Muggles. Dumbledore is away from Hogwarts for long periods, and the Order of the Phoenix has suffered grievous losses. And yet, as in all wars, life goes on.Harry, Ron, and Hermione, having passed their O.W.L. level exams, start on their specialist N.E.W.T. courses. Sixth-year students learn to Apparate, losing a few eyebrows in the process. Teenagers flirt and fight and fall in love. Harry becomes captain of the Gryf

In [8]:
#find all the ratings of the book
review_list = soup.find_all('div', class_ = 'reviewHeader uitext stacked')
review_list

[]

In [39]:
review_list[0].find('a', class_ = 'user', href = True)['href'].split('/')[3].split('-')[0]

In [10]:
# find user information 
user_list = soup.find_all('a', class_ = 'user', href = True)
user_href = [i['href'] for i in soup.find_all('a', class_='user', href=True)]
# user_list
user_href

[]

In [14]:
#find all the ratings of the book
review_list = soup.find_all('div', class_ = 'reviewHeader uitext stacked')

data = []

for r in review_list:
    rev = {
        
        'review': r.find('span', class_ = 'staticStars notranslate').text,
        'book_name': soup.find('title').text.split(sep = 'by')[0] 
        
    }
    data.append(rev)
df = pd.DataFrame(data) 

In [16]:
#review_list = soup.find_all('div', class_ = 'reviewHeader uitext stacked')
def rating(soup, review_list):
    data = []
        
    rev = {'user': r.find('a', class_ = 'user', href = True)['href'].split('/')[3].split('-')[0],
    'review': r.find('span', class_ = 'staticStars notranslate').text}
    
    data.append(rev)
    return data
    

In [ ]:
rating(soup, soup.find_all('div', class_ = 'reviewHeader uitext stacked'))

#### Testing the code

In [305]:
data = []
for r in review_list:
    rev = {
        'user': r.find('a', class_ = 'user', href = True)['href'].split('/')[3].split('-')[0],
        'review': r.find('span', class_ = 'staticStars notranslate').text,
        'book_name': soup.find('title').text.split(sep = 'by')[0]  
    }
    
    data.append(rev)
    

df = pd.DataFrame(data)
df.to_csv('./books.csv')

In [306]:
df.head()

,user,review,book_name
0,111921,it was amazing,Speak
1,9303625,really liked it,Speak
2,30604861,it was amazing,Speak
3,4622890,really liked it,Speak
4,1876075,it was amazing,Speak
